In [40]:
import os
from math import exp
from typing import Dict, List

import tiktoken
from openai import OpenAI

GPT - Cross Encoder

In [4]:
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [8]:
labels = ["Yes", "No"]
MODEL_NAME = "gpt-4o-mini"
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)
print(f"Tokenizer for {MODEL_NAME}: {tokenizer.name}")

Tokenizer for gpt-4o-mini: o200k_base


In [15]:
label_ids = [tokenizer.encode(label) for label in labels]
print(f"Label IDs: {label_ids}")
logit_bias = {id[0]: 1 for id in label_ids if len(id) == 1}
print(f"Logit bias: {logit_bias}")

Label IDs: [[13022], [3160]]
Logit bias: {13022: 1, 3160: 1}


In [14]:
ECOMMERCE_RANKING_PROMPT = '''
You are an Assistant responsible for helping detect whether the retrieved product is relevant to the query. For a given input, you need to output a single token: "Yes" or "No" indicating the retrieved product is relevant to the query.

Query: How to plant a tree?
Product Text: """Cars were invented in 1886, when German inventor Carl Benz patented his Benz Patent-Motorwagen.[3][4][5] Cars became widely available during the 20th century. One of the first cars affordable by the masses was the 1908 Model T, an American car manufactured by the Ford Motor Company. Cars were rapidly adopted in the US, where they replaced horse-drawn carriages.[6] In Europe and other parts of the world, demand for automobiles did not increase until after World War II.[7] The car is considered an essential part of the developed economy."""
Relevant: No

Query: Has the coronavirus vaccine been approved?
Product Text: """The Pfizer-BioNTech COVID-19 vaccine was approved for emergency use in the United States on December 11, 2020."""
Relevant: Yes

Query: What is the capital of France?
Product Text: """Paris, France's capital, is a major European city and a global center for art, fashion, gastronomy and culture. Its 19th-century cityscape is crisscrossed by wide boulevards and the River Seine. Beyond such landmarks as the Eiffel Tower and the 12th-century, Gothic Notre-Dame cathedral, the city is known for its cafe culture and designer boutiques along the Rue du Faubourg Saint-Honoré."""
Relevant: Yes

Query: What are some papers to learn about PPO reinforcement learning?
Product Text: """Proximal Policy Optimization and its Dynamic Version for Sequence Generation: In sequence generation task, many works use policy gradient for model optimization to tackle the intractable backpropagation issue when maximizing the non-differentiable evaluation metrics or fooling the discriminator in adversarial learning. In this paper, we replace policy gradient with proximal policy optimization (PPO), which is a proved more efficient reinforcement learning algorithm, and propose a dynamic approach for PPO (PPO-dynamic). We demonstrate the efficacy of PPO and PPO-dynamic on conditional sequence generation tasks including synthetic experiment and chit-chat chatbot. The results show that PPO and PPO-dynamic can beat policy gradient by stability and performance."""
Relevant: Yes

Query: Explain sentence embeddings
Product Text: """Inside the bubble: exploring the environments of reionisation-era Lyman-α emitting galaxies with JADES and FRESCO: We present a study of the environments of 16 Lyman-α emitting galaxies (LAEs) in the reionisation era (5.8<z<8) identified by JWST/NIRSpec as part of the JWST Advanced Deep Extragalactic Survey (JADES). Unless situated in sufficiently (re)ionised regions, Lyman-α emission from these galaxies would be strongly absorbed by neutral gas in the intergalactic medium (IGM). We conservatively estimate sizes of the ionised regions required to reconcile the relatively low Lyman-α velocity offsets (ΔvLyα<300kms−1) with moderately high Lyman-α escape fractions (fesc,Lyα>5%) observed in our sample of LAEs, indicating the presence of ionised ``bubbles'' with physical sizes of the order of 0.1pMpc≲Rion≲1pMpc in a patchy reionisation scenario where the bubbles are embedded in a fully neutral IGM. Around half of the LAEs in our sample are found to coincide with large-scale galaxy overdensities seen in FRESCO at z∼5.8-5.9 and z∼7.3, suggesting Lyman-α transmission is strongly enhanced in such overdense regions, and underlining the importance of LAEs as tracers of the first large-scale ionised bubbles. Considering only spectroscopically confirmed galaxies, we find our sample of UV-faint LAEs (MUV≳−20mag) and their direct neighbours are generally not able to produce the required ionised regions based on the Lyman-α transmission properties, suggesting lower-luminosity sources likely play an important role in carving out these bubbles. These observations demonstrate the combined power of JWST multi-object and slitless spectroscopy in acquiring a unique view of the early stages of Cosmic Reionisation via the most distant LAEs."""
Relevant: No

Query: {query}
Product Text: """{product_text}"""
Relevant:
'''

In [36]:
def product_relevance(
    client: OpenAI,
    prompt: str,
    model: str = MODEL_NAME,
    temperature: float = 0.0,
    logprobs: bool = True,
    logit_bias: Dict[int, int] = None,
    max_tokens: int = 1,
):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": prompt},
        ],
        temperature=temperature,
        logprobs=logprobs,
        logit_bias=logit_bias,
        max_tokens=max_tokens,
    )

    return response

In [37]:
query = ""
product_text = ""

In [38]:
responses = product_relevance(
    client,
    ECOMMERCE_RANKING_PROMPT.format(query=query, product_text=product_text),
    logit_bias=logit_bias,
)

In [41]:
label = responses.choices[0].message.content
logprob = responses.choices[0].logprobs.content[0].logprob
print(f"Label: {label} with logprob: {logprob} - probability: {exp(logprob)}")

Label: No with logprob: -3.1737043e-06 - probability: 0.9999968263007362
